In [1]:
import numpy as np
import re
import pandas as pd
import datetime
from navec import Navec
from matplotlib import pyplot as pl

# from sklearn.cluster import KMeans
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.metrics.pairwise import euclidean_distances

In [6]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [2]:
path = 'natasha_bd//navec_news_v1_1B_250K_300d_100q.tar'
navec = Navec.load(path)

In [7]:
def get_clean_word(word: str) -> str:
	word = re.sub('[^a-zа-яё-]', '', word, flags=re.IGNORECASE)
	word = word.strip('-')
	return word

In [8]:
def news2emb(news:str) -> np.ndarray:
    """Предложение в эмбеддинг. Универсальная реализация"""
    news_clean = [get_clean_word(word) for word in news.split()]
    embeddings_list = []
    for word in news_clean:
        try:
            embeddings_list.append(navec[word.lower()])
        except KeyError:  # если OUV, эмбеддинг = спецэмбедингу unknown для natasha
            embeddings_list.append(navec['<unk>'])
    news_emb = np.mean(embeddings_list, axis=0)
    return news_emb

In [18]:
def date_news(date):
    """новости на дату в формате YYYY-MM-DD -> dataframe, short_news_list, embeddings"""
    news_df = pd.read_pickle('table_news.pkl')
    news_df['short_date'] = news_df.date.apply(lambda x: x.strftime('%Y-%m-%d'))
    news_df = news_df[news_df.short_date == date]
    list_news = news_df.title[news_df.short_date == date].to_list()
    news_df.drop('short_date', axis=1, inplace=True)
    embeddings = [news2emb(news) for news in list_news]
    return news_df, list_news, embeddings

In [19]:
date_news('2022-07-20')[0]

,date,title,short_news,first_link,raw_news,category,agency
65041,2022-07-20 23:59:18,«Медуза» начнет онлайн 148 дней войны,«Медуза» начнет онлайн 148 дня войны в 8 часов утра по московскому времени. На этом мы приостанавливаем трансляцию с...,NaN,На этом мы приостанавливаем трансляцию событий российского вторжения в Украину. «Медуза» начнет онлайн 148 дня войны...,society,meduzalive
65040,2022-07-20 23:53:09,Фото и видео пожара в Скадовске,Украинские СМИ публикуют фото и видео пожара в Скадовске Херсонской области. Город контролируют российские военные М...,https://t.me/UAonlii/33523,Украинские СМИ публикуют фото и видео пожара в Скадовске Херсонской области. Город контролируют российские военные ...,society,meduzalive
65039,2022-07-20 23:47:32,Белорусский авиадиспетчер тайно записал переговоры во время посадки самолета в Минске,"Белорусский авиадиспетчер тайно записал переговоры во время посадки самолета Ryanair в Минске, в результате которых ...",https://meduza.io/feature/2022/07/20/kakoy-kod-ugrozy-nu-puskay-budet-krasnyy?utm_source=telegram&utm_medium=live&ut...,"«Какой код угрозы? Ну, пускай будет красный». Диспетчер аэропорта тайно записал переговоры во время посадки рейса Ry...",society,meduzalive
65038,2022-07-20 23:38:03,Монатик выступил в Киевском метро,Легендарный украинский музыкант Монатик выступил в Киевском метро. Видео выступления на станции «Майдан Незалежности...,https://t.me/c/1210003725/21408,Украинский музыкант Монатик вернулся на родину и выступил в Киевском метро \n. Видео выступления на станции «Майд...,society,meduzalive
65037,2022-07-20 23:27:59,"На 72-летнюю пенсионерку, прикрепившую антивоенную записку к флагу, составили протокол о «дискредитации армии»","На 72-летнюю пенсионерку, которая прикрепила антивоенную записку к российскому флагу, составили протокол о «дискреди...",https://t.me/ovdinfolive/11669,На 72-летнюю пенсионерку составили протокол о «дискредитации армии» после доноса настоятеля Храма Святого Людовика ...,society,meduzalive
...,...,...,...,...,...,...,...
15002,2022-07-20 08:10:24,Вложения инвесторов в акции упали до рекордного уровня с 2008 года,"По итогам июльского опроса управляющих фондами выяснилось, что вложения инвесторов в акции упали до уровня, который ...",NaN,"BofA: GLOBAL FUND MANAGERS SURVEY – МАКСИМАЛЬНЫЙ РЫНОЧНЫЙ ПЕССИМИЗМ, ОПАСЕНИЯ РЕЦЕССИИ ТАКЖЕ ВЗЛЕТАЮТ ВВЕРХ Итог ию...",economy,russianmacro
15001,2022-07-20 07:41:54,Народный Банк Китая сохранил ставку среднесрочного кредитования на отметке 3.70% годовых,"На фоне резкого падения потребительских цен в Китае Народный Банк Китая (НБК), как и ожидалось, сохранил ставку сред...",NaN,"БАНК КИТАЯ ПО ИТОГАМ СВОЕГО ЗАСЕДАНИЯ ВНОВЬ НЕ СТАЛ МЕНЯТЬ ГОДИЧНУЮ КЛЮЧЕВУЮ СТАВКУ Народный Банк Китая (НБК), как...",economy,russianmacro
15000,2022-07-20 07:22:41,На заокеанских фондовых рынках сохраняется лучшая динамика за последние три недели,На заокеанских фондовых рынках сохраняется лучшая динамика за последние три недели. Наибольший спрос в высокотехноло...,NaN,"🟢 - лучшая динамика за последние три недели на заокеанских рынках, WallStreet показал довольно уверенный рост из-за ...",economy,russianmacro
64939,2022-07-20 00:08:34,Поставки газа в ЕС по «Северному потоку — 1» возобновятся 21 июля,Поставки российского газа в ЕС по газопроводу «Северный поток — 1» возобновятся 21 июля. При этом ЕС планирует ослаб...,NaN,"Итоги сто сорок шестого дня войны \n ➤По данным Reuters, поставки российского газа в ЕС по газопроводу «Северный п...",society,meduzalive


In [20]:
date_df, day_news_list, embeddings = date_news('2022-07-20')

In [22]:
# day_news_list

In [24]:
# embeddings

### Кластеризация лучших четырёх моделей

In [ ]:
# pip install diameter-clustering

In [34]:
import sklearn.neighbors
from diameter_clustering import MaxDiameterClustering, LeaderClustering, QTClustering
from diameter_clustering.dist_matrix import compute_sparse_dist_matrix
from sklearn.cluster import (DBSCAN, OPTICS, AffinityPropagation,
                             AgglomerativeClustering, Birch, KMeans, MeanShift,
                             MiniBatchKMeans)

In [37]:
dist_matrix = compute_sparse_dist_matrix(list(embeddings), metric='cosine')

In [38]:
model_Birch = Birch(n_clusters=None, threshold=0.75, branching_factor=50)
model_AgglomerativeClustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete',
                                        distance_threshold=0.3)
model_MaxDiameterClustering = MaxDiameterClustering(max_distance=0.5, metric=sorted(sklearn.neighbors.VALID_METRICS['brute']), precomputed_dist=True)
model_LeaderClustering = LeaderClustering(max_radius=0.3, metric=sorted(sklearn.neighbors.VALID_METRICS['brute']), precomputed_dist=True)
model_QTClustering = QTClustering(max_radius=0.3, metric=sorted(sklearn.neighbors.VALID_METRICS['brute']),
                             min_cluster_size=2, precomputed_dist=True)

In [40]:
labels_Birch = model_Birch.fit_predict(list(embeddings))
labels_AgglomerativeClustering = model_AgglomerativeClustering.fit_predict(list(embeddings))
labels_MaxDiameterClustering = model_MaxDiameterClustering.fit_predict(compute_sparse_dist_matrix(list(embeddings), metric='cosine'))
labels_LeaderClustering = model_LeaderClustering.fit_predict(compute_sparse_dist_matrix(list(embeddings), metric='cosine'))
labels_QTClustering = model_QTClustering.fit_predict(compute_sparse_dist_matrix(list(embeddings), metric='cosine'))

LeaderClustering fit: 100%|██████████| 164/164 [00:00<00:00, 6067.44it/s]
QTClustering fit. Current cluster size 2.0, total count 20.0:  12%|█▏        | 20.0/165 [00:00<00:00, 2222.15it/s]


In [45]:
def show_clusters(labels):
    """Визуализация работы алгоритмов кластеризации: labels -> pandas.df"""
    df = pd.DataFrame()
    df['text'] = date_df.title
    df['label'] = labels
    df['count'] = df.groupby('label')['label'].transform('count')
    df = df.sort_values(by=['count', 'label'], ascending=[False, True]).loc[:,['text', 'label']]
    num_clusters = len(df.label.value_counts())
    return num_clusters, df

In [44]:
len(date_df)

165

In [49]:
print(f'Birch: {show_clusters(labels_Birch)[0]}')
print(f'Agglomerative: {show_clusters(labels_AgglomerativeClustering)[0]}')
print(f'MaxDiameterClusterin: {show_clusters(labels_MaxDiameterClustering)[0]}')
print(f'LeaderClustering: {show_clusters(labels_LeaderClustering)[0]}')
print(f'QTClustering: {show_clusters(labels_QTClustering)[0]}')


Birch: 159
Agglomerative: 133
MaxDiameterClusterin: 155
LeaderClustering: 155
QTClustering: 9


In [50]:
show_clusters(labels_Birch)[1].head(30)

,text,label
35701,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,149
35700,ЦБ хочет ограничить возможность неквалифицированным инвесторам покупать иностранные акции,149
19063,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,149
65027,Российские власти пытаются заставить «Википедию» удалить статьи о вторжении в Украину,57
6556,Российские власти пытаются заставить «Википедию» удалить статьи о вторжении в Украину. Самое время их прочесть,57
57643,Карлсен отказался защищать титул чемпиона мира по шахматам,92
64997,Магнус Карлсен отказался защищать звание чемпиона мира по шахматам,92
35687,«АвтоВАЗ» выпустит ограниченную партию Lada Largus,103
64965,«АвтоВАЗ» выпустит ограниченную партию Lada Largus,103
57637,«АвтоВАЗ» сделает ограниченную партию Lada Largus из накопленных на складах запчастей,104


In [51]:
show_clusters(labels_AgglomerativeClustering)[1].head(30)

,text,label
35687,«АвтоВАЗ» выпустит ограниченную партию Lada Largus,4
57637,«АвтоВАЗ» сделает ограниченную партию Lada Largus из накопленных на складах запчастей,4
64965,«АвтоВАЗ» выпустит ограниченную партию Lada Largus,4
23513,АвтоВАЗ выпустит 1800 Lada Largus из накопленных на складах компонентов,4
35701,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,40
35700,ЦБ хочет ограничить возможность неквалифицированным инвесторам покупать иностранные акции,40
19063,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,40
35689,ЦБ предлагает запретить неквалифицированным инвесторам сделки с иностранными бумагами,40
65032,"Новак: Россия не будет поставлять нефть на мировые рынки, если страны G7 ограничат цену",0
23541,Испания не одобряет идею Еврокомиссии по ограничению потребления газа на 15%,0


In [52]:
show_clusters(labels_MaxDiameterClustering)[1].head(30)

,text,label
23533,Постпреды ЕС утвердили седьмой пакет санкций против России,16
6555,Германия утвердила седьмой пакет санкций против России,16
19066,В Евросоюзе утвердили седьмой пакет антироссийских санкций,16
57643,Карлсен отказался защищать титул чемпиона мира по шахматам,52
64997,Магнус Карлсен отказался защищать звание чемпиона мира по шахматам,52
10190,Карлсен отказался играть с Непомнящимом за титул чемпиона мира,52
35701,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,1
35700,ЦБ хочет ограничить возможность неквалифицированным инвесторам покупать иностранные акции,1
57644,ЕС готовится к «зиме без российского газа»,8
23524,В ЕС готовятся провести первую зиму без российского газа,8


In [53]:
show_clusters(labels_LeaderClustering)[1].head(30)

,text,label
57643,Карлсен отказался защищать титул чемпиона мира по шахматам,16
64997,Магнус Карлсен отказался защищать звание чемпиона мира по шахматам,16
10190,Карлсен отказался играть с Непомнящимом за титул чемпиона мира,16
23533,Постпреды ЕС утвердили седьмой пакет санкций против России,27
6555,Германия утвердила седьмой пакет санкций против России,27
19066,В Евросоюзе утвердили седьмой пакет антироссийских санкций,27
35700,ЦБ хочет ограничить возможность неквалифицированным инвесторам покупать иностранные акции,1
19063,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,1
64998,В Петербурге прекращено уголовное преследование блогера Юрия Хованского,8
6554,Суд прекратил уголовное дело Юрия Хованского,8


In [58]:
show_clusters(labels_QTClustering)[1].tail(20)

,text,label
35701,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,0
35700,ЦБ хочет ограничить возможность неквалифицированным инвесторам покупать иностранные акции,0
19063,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,0
23533,Постпреды ЕС утвердили седьмой пакет санкций против России,1
6555,Германия утвердила седьмой пакет санкций против России,1
19066,В Евросоюзе утвердили седьмой пакет антироссийских санкций,1
57643,Карлсен отказался защищать титул чемпиона мира по шахматам,2
64997,Магнус Карлсен отказался защищать звание чемпиона мира по шахматам,2
10190,Карлсен отказался играть с Непомнящимом за титул чемпиона мира,2
64998,В Петербурге прекращено уголовное преследование блогера Юрия Хованского,3


### Для бота выбираем агломеративную кластеризацию

In [67]:
def show_date(date):
    """Визуализация работы алгоритмов кластеризации: labels -> pandas.df"""
    date_df, day_news_list, embeddings = date_news(date)
    clast_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='complete',
                                        distance_threshold=0.3)
    labels = clast_model.fit_predict(list(embeddings))
    date_df['label'] = labels
    date_df['count'] = date_df.groupby('label')['label'].transform('count')
    date_df = date_df.sort_values(by=['count', 'label'], ascending=[False, True])
    date_df.drop('count', axis=1, inplace=True)
    return date_df

In [68]:
show_date('2022-07-20')

,date,title,short_news,first_link,raw_news,category,agency,label
35687,2022-07-20 12:41:14,«АвтоВАЗ» выпустит ограниченную партию Lada Largus,"«АвтоВАЗ» выпустит ограниченную партию Lada Largus — не менее 1800 машин, их начнут собирать до конца июля. Автомоби...",vc.ru/transport/466225,"«АвтоВАЗ» выпустит ограниченную партию Lada Largus — не менее 1800 машин, их начнут собирать до конца июля. Автомоб...",technology,vcnews,4
57637,2022-07-20 12:29:08,«АвтоВАЗ» сделает ограниченную партию Lada Largus из накопленных на складах запчастей,«АвтоВАЗ» сделает ограниченную партию Lada Largus из накопленных на складах запчастей. Компания запланировала выпуст...,https://tjournal.ru/news/684880,«АвтоВАЗ» сделает ограниченную партию Lada Largus из накопленных на складах запчастей. Компания запланировала выпус...,technology,TJournal,4
64965,2022-07-20 11:10:48,«АвтоВАЗ» выпустит ограниченную партию Lada Largus,«АвтоВАЗ» выпустит ограниченную партию Lada Largus — из скопившихся на складах запчастей. Ранее компания возобновила...,https://meduza.io/news/2022/07/20/avtovaz-vypustit-ogranichennuyu-partiyu-lada-largus-iz-skopivshihsya-na-skladah-za...,«АвтоВАЗ» выпустит ограниченную партию Lada Largus — из скопившихся на складах запчастей \n В течение почти всего ...,technology,meduzalive,4
23513,2022-07-20 11:05:00,АвтоВАЗ выпустит 1800 Lada Largus из накопленных на складах компонентов,«АвтоВАЗ» выпустит 1800 Lada Largus из накопленных на складах компонентов из-за нехватки комплектующих. К концу лета...,http://vdmsti.ru/gEOn,"«АвтоВАЗ» выпустит 1800 Lada Largus из накопленных на складах компонентов, сообщила пресс-служба компании. В мае п...",technology,vedomosti,4
35701,2022-07-20 19:45:23,ЦБ предложил запретить неквалифицированным инвесторам покупать иностранные акции,В преддверии выборов в Госдуму ЦБ предлагает запретить неквалифицированным инвесторам покупать иностранные акции «ус...,vc.ru/finance/466661,Предложение ЦБ запретить неквалифицированным инвесторам покупать иностранные акции «усилит недоверие частных инвесто...,economy,vcnews,40
...,...,...,...,...,...,...,...,...
4983,2022-07-20 20:13:29,Google приостановит прием на работу на две недели,"Google приостановит прием на работу на две недели, не затрагивая уже сделанные предложения. И это после того как на ...",https://www.theinformation.com/articles/google-announces-hiring-pause,"Google приостановит прием на работу на две недели, не затрагивая уже сделанные предложения. И это после того как на ...",society,addmeto,128
19067,2022-07-20 18:00:21,ЦБ запретил торговлю иностранными акциями для неквалифицированных инвесторов,ЦБ запретил торговлю иностранными акциями для неквалифицированных инвесторов. Продавать акции и закрывать короткие п...,https://cbr.ru/Crosscut/LawActs/File/5955,"Запрет на торговлю иностранными акциями для неквалифицированных инвесторов еще не заработал, но ЦБ уже действует: в ...",society,thebell_io,129
15006,2022-07-20 19:02:19,"Россияне не осознают того, что нам грозит в условиях усиливающейся изоляции",На фоне нарастающих проблем и углубляющегося спада экономики близкий к рекордному оптимизму россиян выглядит иррацио...,https://cbr.ru/Collection/Collection/File/42183/inFOM_22-07.pdf,"ПОТРЕБИТЕЛЬСКИЕ НАСТРОЕНИЯ: ОПТИМИЗМ УСИЛИВАЕТСЯ Опрос населения инФОМ, проводившийся с 29 июня по 8 июля, зафиксир...",economy,russianmacro,130
35699,2022-07-20 18:35:29,МТС купила разработчика сервисов для видеозвонков Webinar,МТС купила разработчика сервисов для видеозвонков Webinar. Сумма сделки могла составить 2 млрд рублей. На её базе МТ...,vc.ru/services/466573,МТС купил разработчика сервисов для видеозвонков Webinar. Сумма сделки могла составить 2 млрд рублей. По собственн...,technology,vcnews,131


In [69]:
final_df = show_date('2022-07-20')

In [70]:
final_df.to_excel('2022-07-20.xlsx')

In [71]:
final_df.category.value_counts()

society          83
economy          44
sports           16
technology       12
entertainment     7
science           3
Name: category, dtype: int64

In [72]:
final_df.agency.value_counts()

meduzalive      37
vedomosti       31
vcnews          18
TJournal        17
thebell_io      17
rbc_sport       15
russianmacro     8
meduzapro        7
rozetked         5
VwordMedia       5
nplusone         3
addmeto          1
ohmypain         1
Name: agency, dtype: int64